# Milestone 3
### 1. What classifiers did you use for milestone 3? (5 points)

1. Gradient Boosting Regressor - Jing Wen
2. Random Forest - Zhixuan(Otto) Hu
3. Extra Tree - XiaoLu He

### 2. Which features did you select for your classifiers? Please comment on the reason for your feature selection. If you choose to work on the bonus question, you can add your features extracted from external datasets at this step. (5 points)

GradientBoostingRegressor: Choose bathrooms, bedrooms, price to predict interest_level, number of bathrooms and bedrooms and price has the strongest relationship with interest_level.

Extra Tree: We selected 'bathrooms', 'bedrooms', 'price', 'latitude', 'longitude' for extra tree classifier since these features have the strongest relationship with interest_level for extra tree classifier.

Random Forest: We choose features: 'price', 'dist_to_city_center','price_per_bedroom', 'price_per_bathroom', 'num_of_features','len_of_description', 'pos_count', 'num_of_photos', 'bedrooms', 'bathrooms'. these feature have relatively high entropy and random forest can process large amount of features so we can use as many as possible.


### 3. How did you perform cross-validation? Please describe the procedure. (10 points)

Use StratifiedKFold allow me to pass a cv parameter to implement k-fold cross validation. In this case, I choose 5-fold cross validation

### 4. What performance did the first version of your classifiers achieve on the validation dataset (in cross-validation) and on the test dataset? Please comment on the performance of the classifier. (15 points: 5 points for performance, and 10 points for comments). 

Extra tree: ~1.21 on Kaggle due to the overfitting. We found the train loss was only ~0.20, but val loss was ~1.20 and it turned out a high score on Kaggle. The score shows an obvious overfitting. 

Random forest: 0.74 on kaggle. We set a max depth of 2 and random state of 2. This turns out to be not very effective. Shallow tree don't do a good job. The classifier has poor accuracy on training data and on kaggle.

GradientBoostingRegressor: ~1.67 on kaggle, compared with train dataset score 0.713 overfit occurs. Mostly due to not handle inf and large Inverse of regularization strength value(C).

### 5. What actions did you take in order to improve your classifiers? You can modify your dataset or the parameters of your classifier. Please record yourmodifications in your report. (30 points: 10 points for each improvement)

Extra Tree: We changed 'max_depth' from 'NONE' to '7' to avoid overfitting. When 'max_depth'='NONE', loss on train data set is ~0.20 and loss on validation data set is ~1.20.

GradientBoostingRegressor: adjusted the 'n_estimators' value to find the optimal one.

Random forest: We removed the depth limitation and added more features. One of Random forest's advantages is that it can have deeper trees without overfitting. Also random forest is able to handle high dimension data.

### 6. How did you check whether any overfitting occurred during your training?Did you observe overfitting? What did you do to avoid overfitting? (10 points)

Compare the performance on trainning data set and validation dataset, if the performance of the model on trainning data set is much better on the validate dataset, it means overfitting occurs.

GradientBoostingRegressor: change the n_estimators parameter of sklearn GradientBoostingRegressor to 1000 from 10000. n_estimators is the number of boosting stages to perform. Gradient boosting is fairly robust to over-fitting so a large number but not as large as 10000 in this case.

Extra Tree: We observed overfitting on our first version of our classifier achieve which have ~0.20 loss on train data and ~1.20 loss on validation data. We adjusted the 'max_depth' value from 'NONE' to '7' to avoid overfitting. 

Random Forest: We adjusted the tree depth and random state. Random forest actually does very well at avoiding overfitting since it uses voting among random trees. I observed very minor overfitting at very large tree depth. Adjusting the random state can reduce overfitting.

### 7. What performance did you achieve on the validation dataset (in cross-validation) and on the test dataset after your modifications? Please, try to explain the gains. (15 points: 5 points for performance, and 10 points for explanation)

Random forest: 0.65. We removed depth limitation and used a very high random state. ultimately achieved around 0.65.

GradientBoostingRegressor: ~0.725. Handle inf value, replace it with mean of the column it belongs to.

Extra tree: ~0.73 on the validation set and ~0.73 on the test dataset after I changed 'max_depth' value to '7'.

### 8. Evaluate one additional evaluation metrics mentioned in class on the validation dataset. Which metric did you use? What were the results? How do these results compare to the results for multi-class logarithmic loss?(10 points)

We evaulated the f score and accuracy score of the classifiers. We found that the lower the log loss the higher the f score and the higher the accuracy score. Our implementations yield around 0.72 for accuracy score and between 0.4 and 0.6 for f score. This is reasonably better than the result we acquired in Milestone 2

### 9. Compare your new classifier with the classifiers used in milestone 2. Try to explain the difference between the performance of the classifiers and the gains of the milestone 3 classifiers. (10 points)

GradientBoostingRegressor： Compared with Logistic Regression, GradientBoostingRegressor builds an additive model by using forward stage-wise way, it allows for the optimization of arbitrary differentiable loss functions. Unlike Logistic Regression, in each stage a regression tree is fit on the negative gradient of the given loss function.

Random Forest: Random forest is built with a "forest" of decision trees. The promise is that it will do better against overfitting. Our results show that indeed random forest overfits much less. In our decision tree implementation, at max tree depth of above 5, the overfitting was very severe. With random forest, even with very high (unlimited) tree depth it's able to achieve high accuracy rate. This is because that the random forest classifier generates a bunch of decision trees using randomly selected data and features instead of just generate one tree with all features and all data points. It runs a sample through all trees and collect outcomes from these trees. This avoids overfitting very effectively.

Extra tree: Extra tree builds an ensemble of unpruned decision or regression trees according to the classical top-down procedure. It splits nodes by choosing cut-points fully at random and it uses the whole learning sample to grow the trees. It has low variance. As we can see, the scores extremely close on train data and validation data. Compare with svm, it is more accurate and efficient.




In [1]:
import pandas as pd
import numpy as np
import re, string, time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
data = pd.read_csv('preprocessed.csv')

# Common Functions

In [3]:
def _preprocess(dtrain, dtest):
    # replace np.inf to np.nan
    dtrain = dtrain.replace([np.inf, -np.inf], np.nan)
    dtest = dtest.replace([np.inf, -np.inf], np.nan)

    # impute np.nan
    dtrain_col_mean = dtrain.mean(axis=0)
    dtrain, dtest = dtrain.fillna(dtrain_col_mean), dtest.fillna(dtrain_col_mean)

    # perform standardization
    dtrain_col_mean, dtrain_col_std = dtrain.mean(axis=0), dtrain.std(axis=0)
    dtrain, dtest = map(lambda x: (x - dtrain_col_mean) / dtrain_col_std, (dtrain, dtest))

    return dtrain, dtest


def _preprocess_log(dtrain, dtest):
    # replace np.inf to np.nan
    dtrain = dtrain.replace([np.inf, -np.inf], np.nan)
    dtest = dtest.replace([np.inf, -np.inf], np.nan)

    # impute np.nan
    dtrain_col_mean = dtrain.mean(axis=0)
    dtrain, dtest = dtrain.fillna(dtrain_col_mean), dtest.fillna(dtrain_col_mean)

    # log transform of min-zero columns
    dtrain_col_min = dtrain.min(axis=0)
    zero_min_index = dtrain_col_min[dtrain_col_min >= 0].index

    dtrain[zero_min_index] = np.log10(dtrain[zero_min_index] + 1.0)
    dtest[zero_min_index] = np.log10(dtest[zero_min_index] + 1.0)

    # perform standardization
    dtrain_col_mean, dtrain_col_std = dtrain.mean(axis=0), dtrain.std(axis=0)
    dtrain, dtest = map(lambda x: (x - dtrain_col_mean) / dtrain_col_std, (dtrain, dtest))

    return dtrain, dtest

def split_data(X, y):
    return train_test_split(X,y)

def run_model(clf, dtrain,dtest=None):
    if dtest:
        clf.fit(dtrain[0],dtrain[1])
        y_train_pred, y_test_pred = clf.predict_proba(dtrain[0]), clf.predict_proba(dtest[0])
        y_train_loss, y_test_loss = log_loss(dtrain[1], y_train_pred), log_loss(dtest[1], y_test_pred)
        return clf, y_train_loss, y_test_loss
    else:
        clf.fit(dtrain[0],dtrain[1])
        y_train_pred = clf.predict_proba(dtrain[0])
        y_train_loss = log_loss(dtrain[1], y_train_pred)
        return clf, y_train_loss

# passing split_data according to features
def train_cv(clf, X, y, preprocess = 'linear'):
    X_train, X_test, y_train, y_test = split_data(X, y)
    # print(X_train, X_test)
    if preprocess == 'log':
        X_train, X_test = _preprocess_log(X_train, X_test)
    elif preprocess == 'linear':
        X_train, X_test = _preprocess(X_train, X_test)
    elif preprocess == 'no_preprocess': 
        #use original data
        pass
        
    cv_scores, n_folds = [], 5
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=816)
   
    for i, (train_ind, val_ind) in enumerate(skf.split(X_train, y_train)):
        print("Running Fold", i + 1, "/", n_folds)
        start = time.time()
        
        train_x, val_x = X_train.iloc[train_ind, :], X_train.iloc[val_ind, :]
        train_y, val_y = y_train.iloc[train_ind], y_train.iloc[val_ind]
        
        clf, train_loss, val_loss = run_model(clf, (train_x, train_y), (val_x, val_y))

        print("train_loss: {0:.6f}, val_loss: {1:.6f}".format(train_loss, val_loss), end="\t")
        
        end = time.time()
        m, s = divmod(end-start, 60)
        h, m = divmod(m, 60)

        print("time elapsed: %d:%02d:%02d" % (h, m, s))
        y_pred = clf.predict(val_x)
        accuracy_score = metrics.accuracy_score(val_y, y_pred)
        f_score = metrics.f1_score(val_y, y_pred, average='macro')
        cv_scores.append([train_loss, val_loss, f_score, accuracy_score])
        
        print("accuracy score: ", accuracy_score)
        print("f score: ", f_score)
        
    mean_train_loss = np.mean([cv_scores[i][0] for i in range(len(cv_scores))])
    mean_val_loss = np.mean([cv_scores[i][1] for i in range(len(cv_scores))])
    
    print("train_loss mean: {0:.6f}, val_loss mean: {1:.6f}".format(mean_train_loss, mean_val_loss))

    return clf, cv_scores



# Training the models

## 1. GradientBoostingRegressor

In [4]:
gb_features = ['bathrooms', 'bedrooms', 'price']
X = data[gb_features]
y = data['interest_level']

In [5]:
gb_clf = GradientBoostingClassifier()
params = {'learning_rate': 0.02,
          'n_estimators': 10000,
          'subsample': 0.7,
          'max_depth': 5,
          'random_state': 36683,
          'verbose': 1
         }
gb_clf.set_params(**params)
train_cv(gb_clf, X, y, preprocess='no_preprocess')

Running Fold 1 / 5
      Iter       Train Loss      OOB Improve   Remaining Time 
         1       13576.8031          23.0494            8.00m
         2       13466.9668          21.4350            7.83m
         3       13473.6977          22.0284            7.83m
         4       13365.9043          19.4552            7.95m
         5       13374.5755          18.5239            7.93m
         6       13416.9872          17.6293            7.91m
         7       13324.9248          16.8784            7.88m
         8       13221.9640          15.8464            7.87m
         9       13261.0183          14.8054            7.84m
        10       13230.4742          14.6534            7.83m
        20       12867.4856           8.7760            7.83m
        30       12741.3353           5.5346            7.75m
        40       12604.9106           3.5317            7.71m
        50       12553.6655           2.5082            7.68m
        60       12506.7989           1.2291      

        20       12922.6931           8.3530            7.67m
        30       12798.4093           5.7352            7.65m
        40       12635.6369           3.5996            7.62m
        50       12481.9753           2.3650            7.60m
        60       12432.1291           1.6707            7.58m
        70       12370.7565           0.9938            7.57m
        80       12310.5703           0.7301            7.55m
        90       12387.9154           0.0537            7.56m
       100       12216.4832           0.3325            7.55m
       200       12116.4898          -0.1302            7.43m
       300       12076.9726          -0.3748            7.32m
       400       11970.8031          -0.3122            7.22m
       500       11829.1061          -0.2682            7.13m
       600       11846.4902          -0.5306            7.06m
       700       11651.9959          -0.5259            6.99m
       800       11729.3979          -0.7391            6.90m
       9

(GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.02, loss='deviance', max_depth=5,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=10000,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=36683, subsample=0.7, tol=0.0001,
                            validation_fraction=0.1, verbose=1,
                            warm_start=False),
 [[0.6286020148340137,
   0.787605508697439,
   0.37314603616017267,
   0.6794538361508452],
  [0.627229294863351,
   0.7865556216002709,
   0.38593561689761,
   0.6840052015604682],
  [0.6294565269737604,
   0.7552119705960043,
   0.38202862805821786,
   0.6831924577373212],
  [0.628157

In [6]:
gb_clf = GradientBoostingClassifier()
params = {'learning_rate': 0.02,
          'n_estimators': 1000,
          'subsample': 0.7,
          'max_depth': 5,
          'random_state': 36683,
          'verbose': 1
         }
gb_clf.set_params(**params)
train_cv(gb_clf, X, y, preprocess='no_preprocess')

Running Fold 1 / 5
      Iter       Train Loss      OOB Improve   Remaining Time 
         1       13581.4371          22.2174           46.96s
         2       13578.5325          21.9653           46.91s
         3       13515.1495          19.6330           47.19s
         4       13473.0160          18.4145           47.56s
         5       13321.8872          17.1375           47.56s
         6       13387.3036          17.2491           47.38s
         7       13253.6058          16.8130           47.24s
         8       13268.9248          14.9846           47.00s
         9       13269.1824          14.1551           47.13s
        10       13166.7659          13.5459           47.02s
        20       12958.7685           9.0291           46.70s
        30       12726.2439           5.5933           45.62s
        40       12682.3492           3.9573           45.26s
        50       12584.3929           2.1353           44.59s
        60       12525.8798           1.5672      

         8       13228.7285          15.2609           46.87s
         9       13214.4378          14.2901           46.91s
        10       13229.9286          14.1058           46.83s
        20       12928.2068           8.9248           46.06s
        30       12825.0995           5.4031           45.62s
        40       12612.6755           3.7239           45.14s
        50       12534.2144           2.4819           44.54s
        60       12413.7606           1.3011           43.91s
        70       12353.5873           0.9131           43.32s
        80       12443.0439           0.6028           42.71s
        90       12329.5988           0.1683           42.18s
       100       12309.8133           0.3419           41.67s
       200       12164.0962          -0.1586           36.64s
       300       12034.5339          -0.3775           31.90s
       400       12034.0795          -0.3783           27.35s
       500       11906.1531          -0.4930           22.76s
       6

(GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.02, loss='deviance', max_depth=5,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=1000,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=36683, subsample=0.7, tol=0.0001,
                            validation_fraction=0.1, verbose=1,
                            warm_start=False),
 [[0.675471621982782,
   0.7233011553639045,
   0.3549329288831719,
   0.6911573472041612],
  [0.6778448981806487,
   0.7170742489944126,
   0.3410952891077786,
   0.6888816644993498],
  [0.6729561289880778,
   0.7308608035195158,
   0.33907459576361076,
   0.6874187256176854],
  [0.67382

## 2. Extra Trees Classifier

In [7]:
et_features = ['bathrooms', 'bedrooms', 'price', 'latitude', 'longitude']
et_X = data[et_features]
et_y = data['interest_level']

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
et_clf = ExtraTreesClassifier()
params = {'max_features': 'sqrt',
              'criterion': 'entropy',
              'max_depth': 7,
              'n_estimators': 2000,
              'bootstrap': True,
              'oob_score': True,
              'n_jobs': -1,
              'verbose': 1,
              'random_state': 36683
              }
et_clf.set_params(**params)

ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=7, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
                     oob_score=True, random_state=36683, verbose=1,
                     warm_start=False)

In [9]:
train_cv(et_clf, et_X, et_y, preprocess='linear')

Running Fold 1 / 5


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    4.2s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:    0.5s
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed:    0.7s
[Parallel(n_job

train_loss: 0.730164, val_loss: 0.734653	time elapsed: 0:00:10


[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed:    0.5s
[Parallel(n_jobs=16)]: Done 2000 out of 2000 | elapsed:    0.5s finished


accuracy score:  0.6890442132639792
f score:  0.27386396266350677
Running Fold 2 / 5


...

train_loss: 0.729481, val_loss: 0.735037	time elapsed: 0:00:10


[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed:    0.5s


accuracy score:  0.6888310843765242
f score:  0.2719424922178364
train_loss mean: 0.729139, val_loss mean: 0.735180


[Parallel(n_jobs=16)]: Done 2000 out of 2000 | elapsed:    0.5s finished


(ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                      criterion='entropy', max_depth=7, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=2000, n_jobs=-1,
                      oob_score=True, random_state=36683, verbose=1,
                      warm_start=False),
 [[0.7301637694208001,
   0.734653365146154,
   0.27386396266350677,
   0.6890442132639792],
  [0.7295829883144259,
   0.7342355482267169,
   0.2748679053394712,
   0.6892067620286085],
  [0.7276553355347098,
   0.7367217472665301,
   0.2734434566196251,
   0.6890442132639792],
  [0.7288097356034714,
   0.7352541411746049,
   0.2802296268524573,
   0.690131685904731],
  [0.7294809979547392,
   0.7350369682957243,
   0.2719424922178364,
   0.6888310843765242

# Random Forest Classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
# rf_features = ['price', 'hour', 'longtitude', 'latitude', 'dist_to_city_center','price_per_bedroom', 'price_per_bathroom', 'num_of_features', 'len_of_description', 'pos_count', 'num_of_photos', 'bedrooms', 'bathrooms', 'kw_quiet_count']
rf_y = data['interest_level']
# rf_X = data.loc[:, data.columns != 'interest_level'].copy()
rf_X = data.drop(['index', 'listing_id', 'interest_level', 'hour'], axis=1).copy()
# rf_X = data[rf_features]
rf_features = rf_X.columns

# rf_X
rf_features


Index(['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price',
       'num_of_photos', 'num_of_features', 'len_of_description',
       'price_per_bedroom', 'price_per_bathroom', 'price_per_bed_bath_room',
       'dist_to_city_center', 'kw_quiet_count', 'kw_new_count',
       'kw_close_count', 'kw_spacious_count', 'kw_convinient_count',
       'kw_safe_count', 'kw_care_count', 'pos_count'],
      dtype='object')

In [12]:
params = {'max_features': 'sqrt',
            'criterion': 'entropy',
            'max_depth': None,
            'n_estimators': 2000,
            'oob_score': True,
            'n_jobs': -1,
            'random_state': 999
            }
rf_clf = RandomForestClassifier()
rf_clf.set_params(**params)

train_cv(rf_clf, rf_X, rf_y, preprocess='linear')

Running Fold 1 / 5
train_loss: 0.169320, val_loss: 0.672262	time elapsed: 0:00:27
accuracy score:  0.7070871261378413
f score:  0.48199723807407335
Running Fold 2 / 5
train_loss: 0.171599, val_loss: 0.655120	time elapsed: 0:00:26
accuracy score:  0.7174902470741222
f score:  0.5062340619687696
Running Fold 3 / 5
train_loss: 0.169470, val_loss: 0.675188	time elapsed: 0:00:25
accuracy score:  0.711963589076723
f score:  0.48464086669305634
Running Fold 4 / 5
train_loss: 0.170065, val_loss: 0.665025	time elapsed: 0:00:25
accuracy score:  0.7146805397496342
f score:  0.49613456784070903
Running Fold 5 / 5
train_loss: 0.171294, val_loss: 0.651890	time elapsed: 0:00:26
accuracy score:  0.7179320435701512
f score:  0.489306663871202
train_loss mean: 0.170350, val_loss mean: 0.663897


(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='entropy', max_depth=None, max_features='sqrt',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=2000,
                        n_jobs=-1, oob_score=True, random_state=999, verbose=0,
                        warm_start=False),
 [[0.16932017247128525,
   0.6722622576793343,
   0.48199723807407335,
   0.7070871261378413],
  [0.17159914564900708,
   0.6551198807050362,
   0.5062340619687696,
   0.7174902470741222],
  [0.16947009103406277,
   0.6751882019733594,
   0.48464086669305634,
   0.711963589076723],
  [0.17006549156800033,
   0.6650246511429032,
   0.49613456784070903,
   0.7146805397496342],
  [0.17129437858921898,
   0.6518896510886163,
   0.48930666387120

# Test data

In [13]:
def testMode(clf, which, features):
    data = pd.read_csv('preprocessed.csv')
    test_data = pd.read_csv('preprocessed_test_data.csv')

    listing_id = test_data['listing_id']
    # test_data = test_data.replace([np.inf, -np.inf], np.nan)
    data, test_data = _preprocess(data, test_data)

    # test_data.info()
    test_data_x = test_data[features]

    predictions = clf.predict_proba(test_data_x)
    # test_data
    predictions
    # res
    submission = pd.DataFrame({'listing_id':listing_id,'low':predictions[:,0], 'medium': predictions[:,1], 'high': predictions[:,2] })
    submission.listing_id = submission.listing_id.astype(int)

    submission.info()
    filename = 'output_' + which + '.csv'
    submission.to_csv(filename,index=False)

In [14]:
testMode(rf_clf, 'rf', rf_features)
# data[rf_features]
# test_data = pd.read_csv('preprocessed_test_data.csv')

# test_data[rf_features]
# rf_features = ['price', 'hour', 'longtitude', 'latitude', 'dist_to_city_center','price_per_bedroom', 'price_per_bathroom', 'num_of_features', 'len_of_description', 'pos_count', 'num_of_photos', 'bedrooms', 'bathrooms', 'kw_quiet_count']
# rf_features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74659 entries, 0 to 74658
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   listing_id  74659 non-null  int32  
 1   low         74659 non-null  float64
 2   medium      74659 non-null  float64
 3   high        74659 non-null  float64
dtypes: float64(3), int32(1)
memory usage: 2.0 MB


In [15]:
testMode(et_clf, 'et', et_features)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.7s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    1.4s
[Parallel(n_jobs=16)]: Done 1218 tasks      | elapsed:    2.3s
[Parallel(n_jobs=16)]: Done 1768 tasks      | elapsed:    3.4s
[Parallel(n_jobs=16)]: Done 2000 out of 2000 | elapsed:    3.8s finished


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74659 entries, 0 to 74658
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   listing_id  74659 non-null  int32  
 1   low         74659 non-null  float64
 2   medium      74659 non-null  float64
 3   high        74659 non-null  float64
dtypes: float64(3), int32(1)
memory usage: 2.0 MB


In [16]:
testMode(gb_clf, 'gb', gb_features)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74659 entries, 0 to 74658
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   listing_id  74659 non-null  int32  
 1   low         74659 non-null  float64
 2   medium      74659 non-null  float64
 3   high        74659 non-null  float64
dtypes: float64(3), int32(1)
memory usage: 2.0 MB
